In [1]:
import os
import rich
import tempfile
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
import openneuro
from lazybids import Dataset
from lazybids.connection import Connection

def create_local_dataset():
    # Create a temporary directory for the local dataset
    # Create the directory ~/.lazybids-test-data if it doesn't exist
    test_data_dir = Path.home() / '.lazybids-test-data'
    dataset_dir = test_data_dir / 'ds005360'
    dataset_dir.mkdir(parents=True, exist_ok=True)
    if not (dataset_dir / 'dataset_description.json').exists():
        # Download the dataset from OpenNeuro if it doesn't exist
        openneuro.download(dataset='ds005360', target_dir=str(dataset_dir))
    
    # Load the dataset using LazyBIDS
    dataset = Dataset.from_folder(str(dataset_dir))
    
    return dataset, str(dataset_dir)

def get_api_dataset():
    # Create a connection to the local server
    connection = Connection("http://localhost:8000")
    
    # Get the dataset from the API
    datasets = connection.list_datasets()
    print(datasets)
    ds005360 = next(ds for ds in datasets if ds['name'] == 'ds005360')
    print(ds005360)
    return connection.get_dataset(ds_id=int(ds005360['id']))

def compare_datasets(local_ds, api_ds):
    # Compare basic dataset information
    assert local_ds.name == api_ds.name, f"Dataset names don't match: {local_ds.name} != {api_ds.name}"
    assert local_ds.bids_version == api_ds.bids_version, f"BIDS versions don't match: {local_ds.bids_version} != {api_ds.bids_version}"
    
    # Compare subjects
    local_subjects = set(local_ds.subjects.keys())
    api_subjects = set(api_ds.subjects.keys())
    assert local_subjects == api_subjects, f"Subject sets don't match: {local_subjects} != {api_subjects}"
    
    # Compare a sample subject
    sample_subject_id = next(iter(local_subjects))
    local_subject = local_ds.subjects[sample_subject_id]
    api_subject = api_ds.subjects[sample_subject_id]
    
    assert local_subject.participant_id == api_subject.participant_id, f"Participant IDs don't match: {local_subject.participant_id} != {api_subject.participant_id}"
    
    # Compare sessions (if any)
    local_sessions = set(local_subject.sessions.keys())
    api_sessions = set(api_subject.sessions.keys())
    assert local_sessions == api_sessions, f"Session sets don't match: {local_sessions} != {api_sessions}"
    
    if local_sessions:
        # Compare a random scan
        import random
        sample_session_id = random.choice(list(local_sessions))
        local_session = local_subject.sessions[sample_session_id]
        api_session = api_subject.sessions[sample_session_id]
        
        assert local_session.session_id == api_session.session_id, f"Session IDs don't match: {local_session.session_id} != {api_session.session_id}"
        
        # Compare scans
        local_scans = set(local_session.scans.keys())
        api_scans = set(api_session.scans.keys())
        assert local_scans == api_scans, f"Scan sets don't match: {local_scans} != {api_scans}"
        
        # Compare a random scan
        sample_scan_id = random.choice(list(local_scans))
        local_scan = local_session.scans[sample_scan_id]
        api_scan = api_session.scans[sample_scan_id]
        
        assert local_scan.name == api_scan.name, f"Scan names don't match: {local_scan.name} != {api_scan.name}"
        assert len(local_scan.files) == len(api_scan.files), f"Number of files don't match: {len(local_scan.files)} != {len(api_scan.files)}"
        assert local_scan.fields == api_scan.fields, f"Scan fields don't match: {local_scan.fields} != {api_scan.fields}"
        assert np.all(local_scan.numpy == api_scan.numpy), f"Scan numpy matrices don't match: {local_scan.numpy} != {api_scan.numpy}"
        if not local_scan.table.empty:
            assert pd.testing.assert_frame_equal(local_scan.table, api_scan.table), f"Scan tables don't match: {local_scan.table} != {api_scan.table}"
        else:
            assert api_scan.table.empty, f"Scan tables don't match: {local_scan.table} != {api_scan.table}"
    # Compare subject-level scans (if any)
    local_subject_scans = set(local_subject.scans.keys())
    api_subject_scans = set(api_subject.scans.keys())
    assert local_subject_scans == api_subject_scans, f"Subject-level scan sets don't match: {local_subject_scans} != {api_subject_scans}"
    if local_subject_scans:
        # Compare a sample subject-level scan
        sample_subject_scan_id = random.choice(list(local_subject_scans))
        local_subject_scan = local_subject.scans[sample_subject_scan_id]
        api_subject_scan = api_subject.scans[sample_subject_scan_id]
        
        assert local_subject_scan.name == api_subject_scan.name, f"Subject-level scan names don't match: {local_subject_scan.name} != {api_subject_scan.name}"
        assert len(local_subject_scan.files) == len(api_subject_scan.files), f"Number of subject-level files don't match: {len(local_subject_scan.files)} != {len(api_subject_scan.files)}"
        assert local_subject_scan.fields == api_subject_scan.fields, f"Subject-level scan fields don't match: {local_subject_scan.fields} != {api_subject_scan.fields}"
        assert np.all(local_subject_scan.numpy == api_subject_scan.numpy), f"Subject-level scan numpy matrices don't match: {local_subject_scan.numpy} != {api_subject_scan.numpy}"
        if not local_subject_scan.table.empty:
            assert pd.testing.assert_frame_equal(local_subject_scan.table, api_subject_scan.table), f"Subject-level scan tables don't match: {local_subject_scan.table} != {api_subject_scan.table}"
        else:
            assert api_subject_scan.table.empty, f"Subject-level scan tables don't match: {local_subject_scan.table} != {api_subject_scan.table}"



/home/roelant/.cache/pypoetry/virtualenvs/lazybids-ui-5inSgEsY-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Creating local dataset...")
local_dataset, temp_dir = create_local_dataset()

print("Getting API dataset...")
api_dataset = get_api_dataset()
print(api_dataset)
print("Comparing datasets...")

compare_datasets(local_dataset, api_dataset)

print("All tests passed successfully!")
os.makedirs(Path.home() / 'Downloads', exist_ok=True)
api_dataset.subjects['sub-02'].sessions['ses-01'].scans['sub-02_ses-01_T1w'].download(Path.home() / 'Downloads')

Creating local dataset...


Loading subjects:   0%|          | 0/66 [00:00<?, ?subject/s]Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-01/./dwi/./sub-01_ses-01_acq-spherical_run-7_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-01/./dwi/./sub-01_ses-01_acq-spherical_run-5_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-01/./dwi/./sub-01_ses-01_acq-linear_run-3_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-01/./dwi/./sub-01_ses-01_acq-spherical_run-8_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-02/./dwi/./sub-01_ses-02_acq-spherical_run-8_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-02/./dwi/./sub-01_ses-02_acq-linear_run-3_dwi.bval
Scan file is not supported: /home/roelant/.lazybids-test-data/ds005360/sub-01/ses-02/./dwi/./sub-01_ses-02_acq-spherical_run-7_dw

Getting API dataset...
[{'OpenNeuroID': 'ds005360', 'name': 'ds005360', 'description': None, 'taskID': '5a218df4-044a-47ac-8047-686f96d9488c', 'folder': '/home/roelant/.lazybids/ds005360-ds005360', 'OpenNeuroVersion': None, 'id': 1, 'icon': '<svg viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg" width="200" height="200">\n<path stroke="transparent" stroke-width="0" fill = "#DEBA9D" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g><circle  transform = "translate(70, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)" ></circle><circle  transform = "translate(30, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)"></circle></g><path transform="translate(-3, -3)" stroke="#000" stroke-width="2" fill = "none" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g id="eye" transform = "translate(50, 50)"><circle id="iris" cx=

In [3]:

connection = Connection("http://localhost:8000")
print(connection.list_datasets())
# Get the dataset from the API
ds = connection.get_dataset(ds_name='ds005360')
print(ds)

[{'OpenNeuroID': 'ds005360', 'name': 'ds005360', 'description': None, 'taskID': '5a218df4-044a-47ac-8047-686f96d9488c', 'folder': '/home/roelant/.lazybids/ds005360-ds005360', 'OpenNeuroVersion': None, 'id': 1, 'icon': '<svg viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg" width="200" height="200">\n<path stroke="transparent" stroke-width="0" fill = "#DEBA9D" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g><circle  transform = "translate(70, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)" ></circle><circle  transform = "translate(30, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)"></circle></g><path transform="translate(-3, -3)" stroke="#000" stroke-width="2" fill = "none" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g id="eye" transform = "translate(50, 50)"><circle id="iris" cx="0" cy="0" r="9" stroke

In [4]:
connection = Connection("http://localhost:8000")

# Get the dataset from the API
datasets = connection.list_datasets()
print(datasets)
ds005360 = next(ds for ds in datasets if ds['name'] == 'ds005360')
print(ds005360)
single_session = connection.get_session(ds005360['id'], 'sub-01', 'ses-01')
print(single_session)


[{'OpenNeuroID': 'ds005360', 'name': 'ds005360', 'description': None, 'taskID': '5a218df4-044a-47ac-8047-686f96d9488c', 'folder': '/home/roelant/.lazybids/ds005360-ds005360', 'OpenNeuroVersion': None, 'id': 1, 'icon': '<svg viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg" width="200" height="200">\n<path stroke="transparent" stroke-width="0" fill = "#DEBA9D" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g><circle  transform = "translate(70, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)" ></circle><circle  transform = "translate(30, 65)" cx="0" cy="0" r="6" fill="rgba(255,255,255,0.4)"></circle></g><path transform="translate(-3, -3)" stroke="#000" stroke-width="2" fill = "none" d="M81.0,66.5Q78,85,59.5,85.5Q41,86,23.0,78.0Q5,70,10.5,51.5Q16,33,27.0,19.0Q38,5,48.0,20.5Q58,36,71.0,42.0Q84,48,81.0,66.5Z" /><g id="eye" transform = "translate(50, 50)"><circle id="iris" cx="0" cy="0" r="9" stroke

In [5]:
print(single_session.scans)

{'sub-01_ses-01_T1w': Scan(name='sub-01_ses-01_T1w', files=[Url('http://localhost:8000/api/dataset/1/subject/sub-01/session/ses-01/scan/sub-01_ses-01_T1w/files/sub-01_ses-01_T1w.nii')], metadata_files=[Url('http://localhost:8000/api/dataset/1/subject/sub-01/session/ses-01/scan/sub-01_ses-01_T1w/files/sub-01_ses-01_T1w.json')], fields={'ITK_FileNotes': 'TE=2.2;Time=155938.962;phase=1', 'ITK_sform_corrected': 'NO', 'aux_file': '', 'bitpix': '16', 'cal_max': '0', 'cal_min': '0', 'datatype': '4', 'descrip': 'TE=2.2;Time=155938.962;phase=1', 'dim[0]': '3', 'dim[1]': '208', 'dim[2]': '256', 'dim[3]': '256', 'dim[4]': '1', 'dim[5]': '1', 'dim[6]': '1', 'dim[7]': '1', 'dim_info': '54', 'intent_code': '0', 'intent_name': '', 'intent_p1': '0', 'intent_p2': '0', 'intent_p3': '0', 'nifti_type': '1', 'pixdim[0]': '0', 'pixdim[1]': '0.94', 'pixdim[2]': '0.941406', 'pixdim[3]': '0.941406', 'pixdim[4]': '2.5', 'pixdim[5]': '0', 'pixdim[6]': '0', 'pixdim[7]': '0', 'qfac': '[UNKNOWN_PRINT_CHARACTERISTIC

In [6]:
print(single_session.scans['sub-01_ses-01_T1w'].files)


[Url('http://localhost:8000/api/dataset/1/subject/sub-01/session/ses-01/scan/sub-01_ses-01_T1w/files/sub-01_ses-01_T1w.nii')]


In [7]:
print(single_session.scans['sub-01_ses-01_T1w'].numpy.shape)

(256, 256, 208)


In [8]:
print(single_session.scans['sub-01_ses-01_T1w'].files)

[PosixPath('/tmp/tmpdkinq5a1/sub-01_ses-01_T1w.nii')]


In [9]:
print(single_session.scans['sub-01_ses-01_T1w'].table)



Empty DataFrame
Columns: []
Index: []


In [10]:
connection = Connection("http://localhost:8000")
ds = connection.get_dataset(4)

In [11]:
print(ds.subjects['sub-01'].sessions['ses-1'].scans)

{'sub-01_ses-1_task-fuzzysemanticrecognition_channels': Scan(name='sub-01_ses-1_task-fuzzysemanticrecognition_channels', files=[], metadata_files=[Url('http://localhost:8000/api/dataset/4/subject/sub-01/session/ses-1/scan/sub-01_ses-1_task-fuzzysemanticrecognition_channels/files/sub-01_ses-1_task-fuzzysemanticrecognition_channels.tsv')], fields={'modality': 'eeg'}, table=      name  type units  low_cutoff  high_cutoff           description  \
0      Fp1   EEG    µV         0.0        100.0  ElectroEncephaloGram   
1      Fp2   EEG    µV         0.0        100.0  ElectroEncephaloGram   
2       Fz   EEG    µV         0.0        100.0  ElectroEncephaloGram   
3       F3   EEG    µV         0.0        100.0  ElectroEncephaloGram   
4       F4   EEG    µV         0.0        100.0  ElectroEncephaloGram   
5       F7   EEG    µV         0.0        100.0  ElectroEncephaloGram   
6       F8   EEG    µV         0.0        100.0  ElectroEncephaloGram   
7      FC1   EEG    µV         0.0        